#### Importando libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time

#### Configurando webscrapper para rolagem infinita de página

In [ ]:
#setting driver
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_argument('--disable-gpu')

#creating driver
s = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.maximize_window()
driver.get("https://www.linkedin.com/jobs/search/?keywords=&location=Brazil&locationId=&geoId=106057199&f_TPR=r86400")

time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

#### Clicando em "See more jobs" ("Ver mais vagas")

In [ ]:
j = 0

while j <= 33:

    timeout = 30
    expand_button_xpath = '/html/body/div[1]/div/main/section/button'
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, expand_button_xpath)))
    driver.execute_script("arguments[0].click();", element)
    
    j += 1
    time.sleep(2)
    

#### Compilando links

In [ ]:
wait = WebDriverWait(driver, 5)

links = []
new_links = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")))
links.extend([l.get_attribute("href") for l in new_links])
links = [x[:-41] for x in links]
print(links)

#### Criando dataframe e inserindo dados

In [ ]:
from tqdm.notebook import tqdm_notebook

df_job = pd.DataFrame() # Creating empty dataframe
j = 0

for i, l in enumerate(tqdm_notebook(links)):
    driver.get(l)
    time.sleep(2) 
    #JOB DESCRIPTION
    try:
        # job_desc = driver.find_elements(By.ID, 'jobDescriptionText')
        job_desc = driver.find_elements(By.CLASS_NAME, 'show-more-less-html__markup')
        for ele in job_desc:
            desc = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        desc = ''
    #JOB TITLE
    try: 
        # job_title = driver.find_elements(By.CLASS_NAME, 'jobsearch-JobInfoHeader-title')
        job_title = driver.find_elements(By.CLASS_NAME, 'topcard__title')
        for ele in job_title:
            title = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        title = ''
    #COMPANY
    try: 
        # job_empresa = driver.find_elements(By.XPATH, "//div[@class='icl-u-lg-mr--sm icl-u-xs-mr--xs']")
        # job_empresa = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/div/div[3]/div/div/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div[1]/div[2]/div')
        job_empresa = driver.find_elements(By.CLASS_NAME, 'topcard__org-name-link')
        for ele in job_empresa:
            empresa = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        empresa = ''
    #LOCATION
    try: 
        # job_localidade = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/div/div[3]/div/div/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div')
        job_localidade = driver.find_elements(By.XPATH, '/html/body/main/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[2]')
        for ele in job_localidade:
            localidade = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        localidade = ''
    #POST DATE
    try: 
        # job_data_publ = driver.find_elements(By.CLASS_NAME, 'jobsearch-HiringInsights-entry--text')
        # job_data_publ = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/div/div[3]/div/div/div[1]/div[1]/div[5]/div[6]/ul/li/span[2]')
        job_data_publ = driver.find_elements(By.CLASS_NAME, 'posted-time-ago__text')
        for ele in job_data_publ:
            data_publ = BeautifulSoup(ele.get_attribute('innerText'), 'html.parser')
    except NoSuchElementException:
        data_publ = ''

    ## creating dictionnary with job data
    dict = {'Plataforma': 'LinkedIn','Titulo': title.text, 'Empresa': empresa.text, 'Localidade': localidade.text, 'Data_Publicacao': data_publ.text.replace('Anunciada ', '').replace('Anunciada: há ', '').replace('Anunciada: ', '').capitalize(), 'Descricao': desc.text, 'Nivel_experiencia': '', 'Tipo_emprego': '', 'Funcao': '', 'Setor': ''}
    ## inserting dictionnary in dataframe
    df_job = df_job.append(dict, ignore_index = True)

In [ ]:
from datetime import datetime
currentDateTime = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
df_job.to_csv(f"./Data/Extracao_LinkedIn_{currentDateTime}.csv", index = False, encoding='utf-8-sig', sep=';')

driver.quit()

In [ ]:
print(df_job)